# Integrating AIS Data With Weather Data

In [1]:
# importing libraries
import pandas as pd

# Import Meteostat library and dependencies
from datetime import datetime
from meteostat import Hourly
from meteostat import Stations
%config Completer.use_jedi = False

In [7]:
data = pd.read_csv('aisdk_20210115.csv')

In [8]:
# dropping unnecessary columns
data = data.drop(columns=['A','B','C','D', 'Destination', 'ETA', 'Draught', 'IMO', 'Callsign'])

# dropping rows with NaN values
data = data.dropna(subset=['SOG', 'COG', 'Heading', 'Latitude', 'Longitude', 'ROT'])

In [9]:
# sample data
df = data[0:20]
df

,# Timestamp,Type of mobile,MMSI,Latitude,Longitude,Navigational status,ROT,SOG,COG,Heading,Name,Ship type,Cargo type,Width,Length,Type of position fixing device,Data source type
0,15/01/2021 00:00:00,Class A,219009229,55.254110,12.374373,Engaged in fishing,0.0,0.0,211.0,211.0,NaN,Undefined,NaN,NaN,NaN,Undefined,AIS
3,15/01/2021 00:00:00,Class A,209869000,55.349473,13.143698,Under way using engine,0.0,12.1,15.8,17.0,NaN,Undefined,NaN,NaN,NaN,Undefined,AIS
4,15/01/2021 00:00:00,Class A,219423000,54.573667,11.924717,Under way using engine,0.0,0.1,336.4,339.0,NaN,Undefined,NaN,NaN,NaN,Undefined,AIS
5,15/01/2021 00:00:00,Class A,257820000,55.018383,12.986552,Restricted maneuverability,0.0,0.8,195.9,303.0,NaN,Undefined,NaN,NaN,NaN,Undefined,AIS
6,15/01/2021 00:00:00,Class A,229882000,54.442888,12.140312,Under way using engine,0.0,11.9,264.0,264.0,NaN,Undefined,NaN,NaN,NaN,Undefined,AIS
8,15/01/2021 00:00:00,Class A,219005921,56.505248,11.759068,Unknown value,0.0,2.8,99.0,106.0,NaN,Undefined,NaN,NaN,NaN,Undefined,AIS
10,15/01/2021 00:00:00,Class A,219005866,57.320545,11.123325,Unknown value,0.0,0.0,189.3,175.0,NaN,Undefined,NaN,NaN,NaN,Undefined,AIS
12,15/01/2021 00:00:00,Class A,219004179,56.788078,8.875468,Under way using engine,0.0,0.0,0.0,85.0,NaN,Undefined,NaN,NaN,NaN,Undefined,AIS
13,15/01/2021 00:00:00,Class A,219000028,55.858678,12.835152,Under way using engine,0.0,0.0,11.4,258.0,NaN,Undefined,NaN,NaN,NaN,Undefined,AIS
15,15/01/2021 00:00:00,Class A,219173000,55.093972,14.690040,Moored,0.0,0.0,225.1,20.0,NaN,Undefined,NaN,NaN,NaN,Undefined,AIS


## Functions to perform weather data fetching

In [10]:
# date negative range generating function
def date_neg(year, month):
    n_month = 0
    n_year = 0
    #2018 01 01
    if month==1:
        n_month = 12
        n_year = year - 1
    else:
        n_month = month - 1
        n_year = year
    return n_year, n_month

# date positive range generating function
def date_pos(year, month):
    p_month = 0
    p_year = 0
    if month==12:
        p_month = 1 
        p_year = year + 1
    else:
        p_month = month + 1
        p_year = year
    return p_year, p_month


# fetch weather data for each row of AIS data
def fetch_weather(row, feature):
    
    dt = datetime.strptime(row['# Timestamp'], '%d/%m/%Y %H:%M:%S')
    
    # Getting the station info from lat, long
    stations = Stations()
    stations = stations.nearby(row['Latitude'], row['Longitude'])
    station = stations.fetch(1)
    
    # Set start and end date
    s_year, s_month = date_neg(dt.year, dt.month)
    e_year, e_month = date_pos(dt.year, dt.month)
    
    # Set time period
    start = datetime(s_year, s_month, dt.day)
    end = datetime(e_year, e_month, dt.day, dt.hour, 0)
    
    # Get hourly data
    data = Hourly(station, start, end)
    try:
        data = data.normalize()
        data = data.interpolate()
        data = data.fetch()
    except:
        data = data.fetch()
    
    date_index = str(dt.year) + str(dt.month).zfill(2) + str(dt.day).zfill(2) + 'T' + str(dt.hour).zfill(2) + '0000'
    return data.loc[date_index, feature]

## Fetching Data For Each Weather Features
- temp: temperature in degree Celcius
- dwpt: dew point temperature in degres Celcius
- rhum: relative humidity
- wdir: wind direction
- wspd: wind speed
- pres: pressure

In [11]:
w_props = ['temp', 'dwpt', 'rhum', 'wdir', 'wspd', 'pres']

# fetch weather for each row of 'df' DataFrame
for prop in w_props:
    df.loc[:, prop] = df.apply(lambda row: fetch_weather(row, prop), axis=1)

c:\users\aungm\appdata\local\programs\python\python38\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
c:\users\aungm\appdata\local\programs\python\python38\lib\site-packages\pandas\core\indexing.py:1675: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [12]:
df

,# Timestamp,Type of mobile,MMSI,Latitude,Longitude,Navigational status,ROT,SOG,COG,Heading,...,Width,Length,Type of position fixing device,Data source type,temp,dwpt,rhum,wdir,wspd,pres
0,15/01/2021 00:00:00,Class A,219009229,55.254110,12.374373,Engaged in fishing,0.0,0.0,211.0,211.0,...,NaN,NaN,Undefined,AIS,-1.7,-4.9,79.0,17.0,11.1,1023.9
3,15/01/2021 00:00:00,Class A,209869000,55.349473,13.143698,Under way using engine,0.0,12.1,15.8,17.0,...,NaN,NaN,Undefined,AIS,-2.3,-5.0,82.0,4.0,24.1,1023.2
4,15/01/2021 00:00:00,Class A,219423000,54.573667,11.924717,Under way using engine,0.0,0.1,336.4,339.0,...,NaN,NaN,Undefined,AIS,NaN,NaN,NaN,NaN,NaN,NaN
5,15/01/2021 00:00:00,Class A,257820000,55.018383,12.986552,Restricted maneuverability,0.0,0.8,195.9,303.0,...,NaN,NaN,Undefined,AIS,-2.3,-5.0,82.0,4.0,24.1,1023.2
6,15/01/2021 00:00:00,Class A,229882000,54.442888,12.140312,Under way using engine,0.0,11.9,264.0,264.0,...,NaN,NaN,Undefined,AIS,1.4,-2.4,76.0,360.0,27.4,1022.6
8,15/01/2021 00:00:00,Class A,219005921,56.505248,11.759068,Unknown value,0.0,2.8,99.0,106.0,...,NaN,NaN,Undefined,AIS,-0.6,-5.8,68.0,14.0,18.5,1024.8
10,15/01/2021 00:00:00,Class A,219005866,57.320545,11.123325,Unknown value,0.0,0.0,189.3,175.0,...,NaN,NaN,Undefined,AIS,-2.4,-5.4,80.0,50.0,16.7,1026.8
12,15/01/2021 00:00:00,Class A,219004179,56.788078,8.875468,Under way using engine,0.0,0.0,0.0,85.0,...,NaN,NaN,Undefined,AIS,-2.9,-4.8,87.0,13.0,3.7,1026.5
13,15/01/2021 00:00:00,Class A,219000028,55.858678,12.835152,Under way using engine,0.0,0.0,11.4,258.0,...,NaN,NaN,Undefined,AIS,-3.3,-5.8,83.0,9.0,13.0,1023.6
15,15/01/2021 00:00:00,Class A,219173000,55.093972,14.690040,Moored,0.0,0.0,225.1,20.0,...,NaN,NaN,Undefined,AIS,0.0,-1.9,87.0,18.0,11.1,1020.0
